In [ ]:
# default_exp data
%load_ext lab_black
# nb_black if running in jupyter
%load_ext autoreload
# automatically reload python modules if there are changes in the
%autoreload 2

In [ ]:
# hide
from nbdev.showdoc import *

# Data

> You should begin your work by cleaning up your data and possibly defining tools for doing it repeateadly.



***input***: raw data

***output***: clean and tidy dataset + toy dataset for testing

***description:***

This is the first notebook of your machine learning project. In this notebook, you will load the data, inspect, clean and make it tidy. 
You will define the data points and their features and labels. The output of this notebook is a clean, tidy dataset ready for analysis and machine learning.
You can also do a basic statistical analysis of the data to better understand it.
For any functions you define for handling the data, remember to mark their cells with `# export` -comment,
so that they will be included in the data.py-module build based on this notebook.
You can also include unit tests for your own functions.

Rewrite this and the other text cells with your own descriptions.

## Import relevant modules

In [ ]:
import numpy as np

# your code here

## Define notebook parameters

Define input, output and additional parameters of this notebook, the information needed for running the notebook.
In your own project, you can do this step in the later iterations of the work,
when you know what is required.
In this cell, only assing values to variables directly: `variable_name = value`.
**Do not derive any information in this cell as it will mess up the parameterization** - do it in the cell below.

In [ ]:
# This cell is tagged with 'parameters'
seed = 0
# your code here

Define any immediate derivative operations from the parameters:

In [ ]:
# set seed
np.random.seed(seed)
# your code here

## Load the data

In [ ]:
# your code here

## Describe the data

Define data points, features and labels

In [ ]:
# your code here

## Clean the data and make it tidy

In [ ]:
# your code here

## Visualize the data

In [ ]:
# your code here

## Intermediate conclusions based on data visualization

what do you see?

## Suffle Dataset

In [ ]:
# your code here

## Save clean and tidy data for further use

In [ ]:
# your code here

## Create small toy dataset for developing and testing the ML methods

In [ ]:
# your code here

save the toy dataset:

In [ ]:
# your code here

## You can now move on to the model notebook!